In [139]:
import os, sys
os.environ['PYTHONPATH'] = '/home/qihao/CS6207'
sys.path.append('/home/qihao/CS6207')
from utils.prosody_utils import *
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

In [140]:
## for Octuple MIDI'
'''
# gen_dir = 'melody_20240406:200213'
# gen_dir = 'melody_20240418:131658' ## oct midi w decoding constraints
gen_dir = 'melody_20240418:124810' ## oct midi wo decoding constraints
dataset_dir = '/data1/qihao/cs6207/octuple/dataset_nopop909'  ## pretrain
# dataset_dir = '/home/qihao/CS6207/dataset'  ## 909
gen_melody_dir = f'/home/qihao/CS6207/octuple/output/{gen_dir}'
gt_melody_dir = f'{dataset_dir}/test'
binary_dir = '/data1/qihao/cs6207/octuple/binary_909'
'''

"\n# gen_dir = 'melody_20240406:200213'\n# gen_dir = 'melody_20240418:131658' ## oct midi w decoding constraints\ngen_dir = 'melody_20240418:124810' ## oct midi wo decoding constraints\ndataset_dir = '/data1/qihao/cs6207/octuple/dataset_nopop909'  ## pretrain\n# dataset_dir = '/home/qihao/CS6207/dataset'  ## 909\ngen_melody_dir = f'/home/qihao/CS6207/octuple/output/{gen_dir}'\ngt_melody_dir = f'{dataset_dir}/test'\nbinary_dir = '/data1/qihao/cs6207/octuple/binary_909'\n"

In [141]:
## for MIDI Rep
'''
gen_dir = 'melody_20240405:203238' ## midi-like rep
dataset_dir = '/home/qihao/CS6207/dataset'  ## 909
gen_melody_dir = f'/home/qihao/CS6207/output_melody/{gen_dir}'
gt_melody_dir = f'{dataset_dir}/test'
binary_dir = '/data1/qihao/cs6207/octuple/binary_909'
'''

"\ngen_dir = 'melody_20240405:203238' ## midi-like rep\ndataset_dir = '/home/qihao/CS6207/dataset'  ## 909\ngen_melody_dir = f'/home/qihao/CS6207/output_melody/{gen_dir}'\ngt_melody_dir = f'{dataset_dir}/test'\nbinary_dir = '/data1/qihao/cs6207/octuple/binary_909'\n"

In [142]:
## for songmass:
gen_melody_dir = "/home/qihao/XAI-Lyricist/songmass/out/midi"
gt_melody_dir = "/home/qihao/XAI-Lyricist/songmass/out/gt"
binary_dir = '/data1/qihao/cs6207/octuple/binary_909'

In [143]:
gen_melodies = glob.glob(os.path.join(gen_melody_dir, '*.mid'))[50:]

In [144]:
event2word_dict, word2event_dict = pickle.load(open(f"{binary_dir}/music_dict.pkl", 'rb'))
test_data = np.load(f"{binary_dir}/words/test_words.npy", allow_pickle=True)
print(test_data[0]['input_path'])

/home/qihao/CS6207/dataset/test/没有你陪伴我真的好孤单_seg0_1_Seg1.mid


In [145]:
for item in test_data:
    src_words, tgt_words = item['src_words'], item['tgt_words']
    print(item['input_path'])
    assert len(src_words)+2 == len(tgt_words)
    for idx in range(len(src_words)):
        strength = word2event_dict['Strength'][src_words[idx]['strength']]
        length = word2event_dict['Length'][src_words[idx]['length']]
        phrase = word2event_dict['Phrase'][src_words[idx]['phrase']]
        bar = word2event_dict['Bar'][tgt_words[idx+1]['bar']]
        pos = word2event_dict['Pos'][tgt_words[idx+1]['pos']]
        token = word2event_dict['Pitch'][tgt_words[idx+1]['token']]
        dur = word2event_dict['Dur'][tgt_words[idx+1]['dur']]
        # print(f"{strength}, {length}, {phrase}; \t {bar}, {pos}, {token}, {dur}")
    break

/home/qihao/CS6207/dataset/test/没有你陪伴我真的好孤单_seg0_1_Seg1.mid


In [146]:
p_tknzr = {
    '<strong>': 0,
    '<substrong>': 1,
    '<weak>': 2,
    '<long>': 3,
    '<short>': 4,
}

all_strength, matched_strength = 0, 0
all_length, matched_length = 0, 0
all_prsd, matched_prsd = 0, 0
matched_note_num = 0
prsd_bleus = []
matched_line_num = 0
print(len(gen_melodies))
for idx, gen_melody in enumerate(gen_melodies):
    gt_melody = os.path.join(gt_melody_dir, os.path.basename(gen_melody)) ## non-songmass
    gt_melody = os.path.join(gt_melody_dir, os.path.basename(gen_melody).split('-')[-1]) ## songmass
    tgt_sent_num = len(miditoolkit.MidiFile(gen_melody).markers)
    is_bound, _, _ = phrasing(gt_melody)
    gt_sent_num = is_bound.count('<true>')
    if tgt_sent_num == gt_sent_num:
        matched_line_num += 1
    else:
        print(tgt_sent_num, gt_sent_num)
    prsd_gen = prosody(gen_melody)
    prsd_gt = prosody(gt_melody)
    # print(prsd_gen, prsd_gt)
    """
    if len(prsd_gen) != len(prsd_gt):
        print(f"| Data {os.path.basename(gen_melody)}:")
    """
    
    if len(prsd_gen) - len(prsd_gt) < 2:
        gt_prsd, gen_prsd = [], []
        matched_note_num += 1
        gt_prsd.extend([p_tknzr[prsd_gt[i][1]] for i in range(len(prsd_gt))])
        gen_prsd.extend([p_tknzr[prsd_gen[i][1]] for i in range(len(prsd_gen))])
        bleu = sentence_bleu([gt_prsd], gen_prsd)
        prsd_bleus.append(bleu)
    # print(abs(len(prsd_gen)-len(prsd_gt)))
    if len(prsd_gen) == len(prsd_gt) or len(prsd_gen) != len(prsd_gt):
        # print(f"{prsd_gen[i][0]==prsd_gt[i][0]}\t{prsd_gen[i][1]==prsd_gt[i][1]}")
        for i in range(min(len(prsd_gen), len(prsd_gt))):
            all_strength += 1
            all_length += 1
            all_prsd += 1
            if prsd_gt[i][0] == prsd_gen[i][0]:
                matched_strength += 1
            if prsd_gt[i][1] == prsd_gen[i][1]:
                matched_length += 1
            if prsd_gt[i][0] == prsd_gen[i][0] and prsd_gt[i][1] == prsd_gen[i][1]:
                matched_prsd += 1
            # print(f"{prsd_gen[i][0]} {prsd_gt[i][0]} \t {prsd_gen[i][1]} {prsd_gt[i][1]}")
    # print()
print(f"| strength: {matched_strength/all_strength*100}%, length: {matched_length/all_length*100}%, prosody: {matched_prsd/all_prsd}%, note_num: {matched_note_num/len(gen_melodies)}, len_num: {matched_line_num/len(gen_melodies)}")
print(f"| prosody BLEU: mean:{np.mean(prsd_bleus)}, max:{np.max(prsd_bleus)}, min:{np.min(prsd_bleus)}")

644
0 6
0 1
0 3
0 13
0 15
0 3
0 12
0 10
0 9
0 12
0 1
0 17
0 19
0 17
0 9
0 20
0 12
0 10
0 9
0 7
0 23
0 12
0 12
0 1
0 2
0 27
0 9
0 13
0 5
0 9
0 11
0 7
0 21
0 4
0 24
0 21
0 3
0 21
0 32
0 19
0 7
0 12
0 18
0 2
0 11
0 13
0 12
0 16
0 21
0 4
0 18
0 8
0 5
0 8
0 4
0 10
0 3
0 27
0 12
0 33
0 28
0 10
0 9
0 15
0 9
0 7
0 24
0 9
0 3
0 17
0 32
0 10
0 4
0 24
0 15
0 6
0 2
0 30
0 8
0 13
0 12
0 15
0 18
0 11
0 6
0 12
0 6
0 25
0 3
0 10
0 7
0 11
0 5
0 10
0 12
0 8
0 10
0 18
0 2
0 17
0 16
0 19
0 10
0 17
0 4
0 1
0 1
0 16
0 12
0 18
0 3
0 2
0 18
0 1
0 8
0 18
0 7
0 35
0 8
0 8
0 26
0 10
0 10
0 11
0 1
0 6
0 3
0 14
0 8
0 5
0 5
0 14
0 17
0 14
0 11
0 23
0 13
0 17
0 18
0 20
0 2
0 12
0 12
0 9
0 6
0 1
0 2
0 4
0 12
0 9
0 3
0 7
0 8
0 5
0 2
0 11
0 20
0 1
0 17
0 26
0 6
0 6
0 13
0 18
0 8
0 16
0 20
0 14
0 23
0 8
0 18
0 19
0 18
0 8
0 17
0 20
0 7
0 15
0 20
0 1
0 14
0 1
0 5
0 6
0 20
0 21
0 3
0 3
0 9
0 18
0 8
0 19
0 1
0 1
0 21
0 13
0 4
0 1
0 15
0 1
0 7
0 1
0 4
0 4
0 12
0 16
0 14
0 1
0 1
0 18
0 4
0 8
0 7
0 3
0 2
0 5
0 10
0 12
0 5
0 1

In [125]:
53/60

0.8833333333333333